In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

def animar_e_salvar_mp4(caminho_arquivo_entrada):
    """
    Lê os dados da simulação Fortran, gera uma animação e salva como MP4
    na mesma pasta do arquivo de entrada.
    """
    
    # --- 1. Gerenciamento de Caminhos e Arquivos ---
    # Obtém o caminho absoluto para evitar erros
    caminho_absoluto = os.path.abspath(caminho_arquivo_entrada)
    
    # Descobre a pasta onde o arquivo está
    pasta_do_arquivo = os.path.dirname(caminho_absoluto)
    
    # Cria o nome do arquivo de saída (mesmo nome, extensão .mp4)
    nome_base = os.path.splitext(os.path.basename(caminho_absoluto))[0]
    caminho_saida = os.path.join(pasta_do_arquivo, f"{nome_base}.mp4")
    
    print(f"Lendo dados de: {caminho_absoluto}")
    
    # --- 2. Carregamento dos Dados ---
    try:
        # O código Fortran usa vírgula como separador 
        data = np.loadtxt(caminho_absoluto, delimiter=',')
    except OSError:
        print(f"Erro: Arquivo não encontrado em {caminho_absoluto}")
        return

    tempo = data[:, 0]
    pos_x = data[:, 1]
    pos_y = data[:, 2]
    
    # --- 3. Configuração do Gráfico ---
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Ajuste os limites conforme sua simulação (x inicial é 1.0) [cite: 1]
    limite = 1.5
    ax.set_xlim(-limite, limite)
    ax.set_ylim(-limite, limite)
    ax.set_aspect('equal') # Importante para a órbita ser redonda
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.set_xlabel('X (UA)')
    ax.set_ylabel('Y (UA)')
    ax.set_title(f'Simulação: {nome_base}')

    # Elementos fixos e móveis
    ax.plot(0, 0, 'yo', markersize=12, label='Sol') # Centro (0,0)
    planeta, = ax.plot([], [], 'bo', markersize=6, label='Planeta')
    rastro, = ax.plot([], [], 'b-', linewidth=1, alpha=0.5)
    texto_tempo = ax.text(0.05, 0.95, '', transform=ax.transAxes)
    ax.legend(loc='upper right')

    # --- 4. Lógica da Animação ---
    
    # Para performance e tamanho do vídeo, pulamos passos.
    # imax=10000 é muito para vídeo, vamos pegar 1 a cada 10 ou 20 pontos.
    passo_frame = 10 
    indices_frames = range(0, len(tempo), passo_frame)

    def init():
        planeta.set_data([], [])
        rastro.set_data([], [])
        texto_tempo.set_text('')
        return planeta, rastro, texto_tempo

    def update(frame_idx):
        # Pega os dados atuais
        x = pos_x[frame_idx]
        y = pos_y[frame_idx]
        
        # Atualiza planeta
        planeta.set_data([x], [y])
        
        # Atualiza rastro (desenha do início até o frame atual)
        rastro.set_data(pos_x[:frame_idx], pos_y[:frame_idx])
        
        # Atualiza texto
        texto_tempo.set_text(f'Tempo: {tempo[frame_idx]:.2f} anos')
        
        return planeta, rastro, texto_tempo

    # Cria a animação
    anim = FuncAnimation(fig, update, frames=indices_frames, init_func=init, blit=True)

    # --- 5. Salvando o Arquivo ---
    print("Gerando vídeo (isso pode levar alguns segundos)...")
    
    # Configurações do codificador MP4
    writer = FFMpegWriter(fps=30, metadata=dict(artist='SimulacaoPython'), bitrate=1800)
    
    try:
        anim.save(caminho_saida, writer=writer)
        print(f"Sucesso! Arquivo salvo em: {caminho_saida}")
    except Exception as e:
        print(f"Erro ao salvar o vídeo. Verifique se o FFMPEG está instalado.\nDetalhe: {e}")
    
    plt.close(fig) # Fecha a janela para não travar o script

# --- Exemplo de Uso ---
if __name__ == "__main__":
    # Basta passar o nome do arquivo gerado pelo Fortran
    animar_e_salvar_mp4('saida.txt')

Lendo dados de: /home/joliveira/Documents/IntroFiscomp2025/Projeto-5/tarefa-A/saida.txt
Gerando vídeo (isso pode levar alguns segundos)...
Sucesso! Arquivo salvo em: /home/joliveira/Documents/IntroFiscomp2025/Projeto-5/tarefa-A/saida.mp4
